### Grouping by cloudiness

Let's group Sentinel-2 images by how cloudy they are, and composite them into low-cloud and high-cloud composites. We'll look at a spot renowned for its clouds: Seattle.

`groupby` works on more than just dates; you can group by any metadata value on an Image.

You could even use it to group by `cloud_fraction`.
Though deriving any possible practical use of this is left as an exercise to the reader, it's still kind of fun to look at.

In [ ]:
import descarteslabs.workflows as wf

In [30]:
wf.map.center = [47.6857302113159, -122.19406127929689]
wf.map.zoom = 9

[47.6857302113159, -122.19406127929689]

In [29]:
wf.map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart Jupyter and re-run this notebook.


In [3]:
s2 = wf.ImageCollection.from_id('sentinel-2:L1C', start_datetime="2019-01-01")

In [4]:
grouped = s2.groupby(lambda img: img.properties['cloud_fraction'] // 0.25)
# cloud_fraction is 0-1, so this should give us up to 5 groups

In [5]:
low = grouped.groups[0].median(axis="images")
high = grouped.groups[3].mean(axis="images")
# notice we use `median` on clear images, `mean` on cloudy

In [6]:
low.pick_bands('red green blue').visualize('low', scales=[[0, 0.4], [0, 0.4], [0, 0.4]])
high.pick_bands('red green blue').visualize('high', scales=[[0, 1], [0, 1], [0, 1]])

-------

How many images fall into each group?

In [27]:
group_lengths = grouped.groups.items().map(
    lambda group_imgs: (group_imgs[0], group_imgs[1].length())
)

In [28]:
group_lengths.compute(wf.map.geocontext())

[######] | Steps: 0/0 | Stage: DONE | Status: SUCCESS                         

[(0.0, 51), (1.0, 22), (2.0, 26), (3.0, 84), (4.0, 5)]

What fraction of images have `cloud_fraction` <= 0.25?

In [23]:
clear_days_fraction = group_lengths[0] / (group_lengths.reduce(lambda accum, x: accum + x))

In [25]:
clear_days_fraction.compute(wf.map.geocontext())

[######] | Steps: 0/0 | Stage: DONE | Status: SUCCESS                         

0.2712765957446808

Indeed, it's cloudy a lot in Seattle.